In [11]:
import cv2
import numpy as np

# Dense optical flow
"""Dense optical flow attempts to compute the optical flow vector for
every pixel of each frame. While such computation may be slower, it 
gives a more accurate result and a denser result suitable for 
applications such as learning structure from motion and video 
segmentation."""

# HSV colorspace - hue, saturation, value
"""Hue channel models color type and thus is very useful in image 
processing tasks that need to segment objects based on its color. 
Vairiation of saturation goes from unsaturated to represent shades of
gray and fully saturated (no white component). Value channel describes 
brightness of intensity of color."""

# cap = cv2.VideoCapture('videos/car.mp4')
cap = cv2.VideoCapture(0)

ret, first_frame = cap.read()

prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

mask = np.zeros_like(first_frame)
# print(mask)

mask[..., 1] = 255


while(cap.isOpened()):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Computes a dense optical flow using the Gunnar Farneback’s algo
    # Gunnar Farneback Optical Flow
    """In dense optical flow, we look at all of points(unlike Lucas 
    Kanade which works only on corner points detected by Shi-Tomasi 
    Algorithm) and detect pixel intensity changes between two frames,
    resulting in an image with highlighted pixels, after converting to
    hsv format for clear visibility.
    It computes the magnitude and direction of optical flow from array
    of flow vectors, i.e., (dx/dt, dy/dt). Later it visualizes angle 
    (direction) of flow by hue and the distance (magnitude) of flow by
    value of HSV color representation.For visibility to be optimal, 
    strength of HSV is set to 255. """
    
    """Parameters: 
    prev – first 8-bit single-channel input image.
    next – second input image of the same size and the same type as 
    prev.
    flow – computed flow image that has the same size as prev and type
    CV_32FC2.
    pyr_scale – parameter, specifying the image scale (<1) to build 
    pyramids for each image; pyr_scale=0.5 means a classical pyramid,
    where each next layer is twice smaller than the previous one.
    levels – number of pyramid layers including the initial image; 
    levels=1 means that no extra layers are created and only original
    images are used.
    winsize – averaging window size; larger values increase algorithm
    robustness to image noise and give more chances for fast motion 
    detection, but yield more blurred motion field.
    iterations – number of iterations the algorithm does at each 
    pyramid level.
    poly_n – size of the pixel neighborhood used to find polynomial 
    expansion in each pixel; larger values mean that the image will be
    approximated with smoother surfaces, yielding more robust algorithm
    and more blurred motion field, typically poly_n =5 or 7.
    poly_sigma – standard deviation of the Gaussian that is used to
    smooth derivatives used as a basis for the polynomial expansion;
    for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, good value
    would be poly_sigma=1.5.
    flags –"""
    flow = cv2.calcOpticalFlowFarneback(prev_gray,
                                       gray,
                                       None,
                                       0.5,
                                       3,
                                       15,
                                       3, 
                                       5,
                                       1.5,
                                       0)
    
    # Calculates the magnitude and angle of 2D vectors.
    magn, angle = cv2.cartToPolar(flow[..., 0],
                                 flow[..., 1])
    
    # Set image hue depending on optical flow direction
    mask[..., 0] = angle*180/np.pi/2
    
    # Normalize magnitude
    mask[..., 2] = cv2.normalize(magn,
                                None,
                                0,
                                255,
                                cv2.NORM_MINMAX)
    
    # Convert hsv to rgb
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2RGB)
    
    cv2.imshow('Dense Optical Flow', np.hstack([frame, rgb]))
    prev_gray = gray
    
    # Press esc to exit
    if cv2.waitKey(10) & 0xFF ==27:
        break
        
cap.release()
cv2.destroyAllWindows()
    